In [43]:
import requests
import zipfile
import os

url = "https://static.nhtsa.gov/odi/ffdd/inv/FLAT_INV.zip"
metadata_url = "https://static.nhtsa.gov/odi/ffdd/inv/INV.txt"
output_path = "investigations/FLAT_INV.zip"
extracted_folder = "investigations"  # Specify the folder where you want to extract the files

# Ensure the directory exists
os.makedirs("investigations", exist_ok=True)

# Download the ZIP file
r = requests.get(url)
with open(output_path, "wb") as f:
    f.write(r.content)


print(f"File downloaded successfully to {output_path}")

# Extract the contents
with zipfile.ZipFile(output_path, "r") as zip_ref:
    zip_ref.extractall(extracted_folder)

print(f"Files extracted to {extracted_folder}")

# Download the metadata file
metadata_r = requests.get(metadata_url)
with open("investigations/INV.txt", "wb") as f:
    f.write(metadata_r.content)

print(f"Metadata file downloaded successfully")

import re

# Read the content of INV.txt
with open("investigations/INV.txt", 'r') as f:
    metadata = f.read()

# Extract the table using regex
table_pattern = re.compile(r'Field#\s+Name\s+Type/Size\s+Description\s+------\s+----------\s+---------\s+-------------------------------\s+(.*)', re.DOTALL)
match = table_pattern.search(metadata)

if match:
    table_text = match.group(1).strip()
    lines = table_text.split("\n")
    table_data = []
    
    for line in lines:
        columns = re.split(r'\s{2,}', line.strip())  # Split by 2 or more spaces
        if len(columns) == 4:
            field_number, name, type_size, description = columns
            table_data.append({
                "Field#": field_number,
                "Name": name,
                "Type/Size": type_size,
                "Description": description
            })
    
columns = [x['Name'] for x in table_data]

import pandas as pd
from config import *
investigations = pd.read_csv('investigations/FLAT_INV.txt',sep='\t',header=None,names=columns,dtype=str)
db = pg_connect()
investigations.to_sql('investigations',db,index=False,if_exists = 'replace')
db.dispose()

File downloaded successfully to investigations/FLAT_INV.zip
Files extracted to investigations
Metadata file downloaded successfully


In [40]:
investigations

,NHTSA ACTION NUMBER,MAKE,MODEL,YEAR,COMPNAME,MFR_NAME,ODATE,CDATE,CAMPNO,SUBJECT,SUMMARY
0,AQ08001,PACE AMERICAN,TRAILER,2003,WHEELS,"PACE AMERICAN, INC.",20080618,20081029,NaN,PACE AMERICAN 573 RETRACTION,"BY LETTER DATED NOVEMBER 8, 2007, PACE AMERICA..."
1,AQ08001,PACE AMERICAN,TRAILER,2004,WHEELS,"PACE AMERICAN, INC.",20080618,20081029,NaN,PACE AMERICAN 573 RETRACTION,"BY LETTER DATED NOVEMBER 8, 2007, PACE AMERICA..."
2,AQ08001,PACE AMERICAN,TRAILER,2005,WHEELS,"PACE AMERICAN, INC.",20080618,20081029,NaN,PACE AMERICAN 573 RETRACTION,"BY LETTER DATED NOVEMBER 8, 2007, PACE AMERICA..."
3,AQ08001,PACE AMERICAN,TRAILER,2006,WHEELS,"PACE AMERICAN, INC.",20080618,20081029,NaN,PACE AMERICAN 573 RETRACTION,"BY LETTER DATED NOVEMBER 8, 2007, PACE AMERICA..."
4,AQ09001,APEXCONE,APX9004,9999,EXTERIOR LIGHTING,CARFILO,20090326,20090706,05E069000,HID REPLACEMENT KIT RECALL CAMPAIGNS,RMD IDENTIFIED SEVERAL HID REPLACEMENT LIGHTIN...
...,...,...,...,...,...,...,...,...,...,...,...
152671,TA92007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,92V105000,TA,There is no summary currently available
152672,TA92008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,92V138000,TA,There is no summary currently available
152673,TA92009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,92V144000,TA,There is no summary currently available
152674,TA92011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,92V169000,TA,There is no summary currently available


In [38]:
table_data

[{'Field#': '1',
  'Name': 'NHTSA ACTION NUMBER',
  'Type/Size': 'CHAR(10)',
  'Description': 'NHTSA Identification Number'},
 {'Field#': '2',
  'Name': 'MAKE',
  'Type/Size': 'CHAR(25)',
  'Description': 'Vehicle/Equipment Make'},
 {'Field#': '3',
  'Name': 'MODEL',
  'Type/Size': 'CHAR(256)',
  'Description': 'Vehicle/Equipment Model'},
 {'Field#': '4',
  'Name': 'YEAR',
  'Type/Size': 'CHAR(4)',
  'Description': 'Model Year,'},
 {'Field#': '5',
  'Name': 'COMPNAME',
  'Type/Size': 'CHAR(256)',
  'Description': 'Component Description'},
 {'Field#': '6',
  'Name': 'MFR_NAME',
  'Type/Size': 'CHAR(40)',
  'Description': "Manufacturer's Name"},
 {'Field#': '7',
  'Name': 'ODATE',
  'Type/Size': 'CHAR(8)',
  'Description': 'Date Opened (YYYYMMDD)'},
 {'Field#': '8',
  'Name': 'CDATE',
  'Type/Size': 'CHAR(8)',
  'Description': 'Date Closed (YYYYMMDD)'},
 {'Field#': '9',
  'Name': 'CAMPNO',
  'Type/Size': 'CHAR(9)',
  'Description': 'Recall Campaign Number, if applicable'},
 {'Field#': '1

In [15]:
import requests
import zipfile
import os

url = "https://static.nhtsa.gov/odi/ffdd/inv/FLAT_INV.zip"
metadata_url = "https://static.nhtsa.gov/odi/ffdd/inv/INV.txt"
output_path = "/investigations/FLAT_INV.zip"
extracted_folder = "/investigations"  # Specify the folder where you want to extract the files

# Download the ZIP file
r = requests.get(url)
with open(output_path, "wb") as f:
    f.write(r.content)

print(f"File downloaded successfully to {output_path}")

# Download the metadata file
metadata_r = requests.get(metadata_url)
with open("/investigations/INV.txt", "wb") as f:
    f.write(metadata_r.content)

print(f"Metadata file downloaded successfully")

# Extract the contents
with zipfile.ZipFile(output_path, "r") as zip_ref:
    zip_ref.extractall(extracted_folder)

print(f"Files extracted to {extracted_folder}")


File downloaded successfully to /investigations/FLAT_INV.zip
Metadata file downloaded successfully
Files extracted to /investigations


In [4]:
import requests
import zipfile
import os

url = "https://static.nhtsa.gov/odi/ffdd/inv/FLAT_INV.zip"
metadata_url = "https://static.nhtsa.gov/odi/ffdd/inv/INV.txt"
output_path = "FLAT_INV.zip"
extracted_folder = "investigations"  # Specify the folder where you want to extract the files

# Download the ZIP file
r = requests.get(url)
with open(output_path, "wb") as f:
    f.write(r.content)

print(f"File downloaded successfully to {output_path}")

# Extract the contents
with zipfile.ZipFile(output_path, "r") as zip_ref:
    zip_ref.extractall(extracted_folder)

print(f"Files extracted to {extracted_folder}")


File downloaded successfully to FLAT_INV.zip
Files extracted to investigations


In [9]:
import pandas as pd
data = pd.read_csv('investigations/FLAT_INV.txt',sep='\t',header=None)

In [10]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10
0,AQ08001,PACE AMERICAN,TRAILER,2003.0,WHEELS,"PACE AMERICAN, INC.",20080618.0,20081029.0,NaN,PACE AMERICAN 573 RETRACTION,"BY LETTER DATED NOVEMBER 8, 2007, PACE AMERICA..."
1,AQ08001,PACE AMERICAN,TRAILER,2004.0,WHEELS,"PACE AMERICAN, INC.",20080618.0,20081029.0,NaN,PACE AMERICAN 573 RETRACTION,"BY LETTER DATED NOVEMBER 8, 2007, PACE AMERICA..."
2,AQ08001,PACE AMERICAN,TRAILER,2005.0,WHEELS,"PACE AMERICAN, INC.",20080618.0,20081029.0,NaN,PACE AMERICAN 573 RETRACTION,"BY LETTER DATED NOVEMBER 8, 2007, PACE AMERICA..."
3,AQ08001,PACE AMERICAN,TRAILER,2006.0,WHEELS,"PACE AMERICAN, INC.",20080618.0,20081029.0,NaN,PACE AMERICAN 573 RETRACTION,"BY LETTER DATED NOVEMBER 8, 2007, PACE AMERICA..."
4,AQ09001,APEXCONE,APX9004,9999.0,EXTERIOR LIGHTING,CARFILO,20090326.0,20090706.0,05E069000,HID REPLACEMENT KIT RECALL CAMPAIGNS,RMD IDENTIFIED SEVERAL HID REPLACEMENT LIGHTIN...
